## Project - Scrape and Analyze Data Analyst Job Requirements with Python

We will use Python to perform Web Scraping on a Job Posting Site using Python.
Objectives:
- Increase the efficiency of job vacancy sourcing,
- Improve the quality of job vacancy sourcing, and 
- Gain a competitive advantage

We will analyze the data using web scraping tools that can automatically extract job posting data from multiple job posting sites through Extract, Transform and Load process.  We will then use the analysis to provide a more efficient way to provide job vacancies to better serve clients. The feature will help the recruitment agency by getting relevant openings to their clients more quickly, giving their clients a competitive advantage over other applicants.

### The Challenge

Conduct web scraping analysis to automatically extract job posting data from a job posting site. To do this, we will set up the environment, identify the job posting site, scrape the data, process, analyze and visualize the data. We are at liberty to choose whichever major job posting website we prefer.
Thereafter, we will perform our analysis and share findings

## Importing the required Libraries

In [ ]:
# Data scraping

from bs4 import BeautifulSoup
import requests
import urllib.parse

# Data manipulation
import pandas as pd
import numpy as np

# Import visualization Libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline

# Appl styling
#plt.style.use("ggplot")
#rcParams['figure.figsize'] = (12, 6)
#sns.set_style('darkgrid')
#matplotlib.rcParams['font.size'] = 14
#matplotlib.rcParams['figure.figsize'] = (9, 5)
#matplotlib.rcParams['figure.facecolor'] = '#00000000' 

### Extraction Process

In [ ]:
# Exracting function
def extract(page):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0'}
    weburl = f'https://ai-jobs.net/?cat=12&key=&exp=&sal={page}'
    r = requests.get(weburl, headers)
    #r.status_code #The code gives the vaklue 200 when executed. This shows that the request is successful
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup 


### Transform Process

In [ ]:
# Transfroming function
def transform(soup):
    divs = soup.find_all('a', class_ = 'col pt-2 pb-3')
    
    # extracting and transforming the data from the website into a dictionary
    for item in divs:
        title = item.find('h2').text if item.find('h2') else "N/A"
        company = item.find('p').text if item.find('p') else "N/A"
        location = item.find('span', class_='d-none d-md-block text-break mb-1').text if item.find('span', class_='d-none d-md-block text-break mb-1') else "N/A"
        salary = item.find('span', class_='badge rounded-pill text-bg-success d-none d-md-inline-block').text if item.find('span', class_='badge rounded-pill text-bg-success d-none d-md-inline-block') else "N/A"
        experience = item.find('span', class_='badge rounded-pill text-bg-info my-md-1 d-none d-md-inline-block').text if item.find('span', class_='badge rounded-pill text-bg-info my-md-1 d-none d-md-inline-block') else "N/A"
        job_type = item.find('span', class_='badge rounded-pill text-bg-secondary my-md-1 ms-1').text if item.find('span', class_='badge rounded-pill text-bg-secondary my-md-1 ms-1') else "N/A"
        skills = [skill.text for skill in item.find_all('span', class_='badge rounded-pill text-bg-light')] 
        benefits_tags = item.find_all('span', class_ = 'badge rounded-pill text-bg-success')
        benefits = [benefit.text for benefit in benefits_tags] if benefits_tags else [ ]

        job = {
            'Title' : title,
            'Company' : company,
            'Salary': salary,
            'Experience': experience,
            'Job_type' : job_type,
            'Skills' : ', '.join(skills).strip('[ ]'),
            'Benefits' : ', '.join(benefits)
        }
        joblist.append(job)
    return


In [ ]:
# create an empty list (dataframe) where the extracted and transformed data will be loaded.
joblist = []

for i in range (0, 40, 10):
    print(f'Getting page, {i}')
    c = extract(0)
    transform(c)

### Loading Process

In [ ]:
# Move the data into a dataframe
import pandas as pd
df = pd.DataFrame(joblist)


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
# Create a csv file that hold the data
df.to_csv('Analyst_jobs.csv')

In [ ]:
### Understanding the Data

In [ ]:
df.isnull().sum

In [ ]:
# dropping null value columns to aavoid errors
df.dropna(inplace = True)
df.head()

In [ ]:
# new dataframe with split value columns
import pandas as pd
new = df["Salary"].str.split(" ", n = 4, expand = True)
new

In [ ]:
# Making separate first name column from new dataframe
df['Currency'] = new[0]
df['Min Salary'] = new[1]
df['Max Salary'] = new[3]
df.head()

In [ ]:
df["Lower_Salary"] = df["Min Salary"].str.strip("kK").astype(float)
df["Upper_Salary"] = df["Max Salary"].str.strip("kK").astype(float)
df.tail()

In [ ]:
df.drop(columns = ["Min Salary", "Max Salary"], inplace = True)
df.head()

In [ ]:
df = df.convert_dtypes()
df.dtypes

In [ ]:
df['Avg_Salary(e3)'] = (df['Lower_Salary'] + df['Upper_Salary']) / 2
df.tail()

In [ ]:

# Currency exchange rates (USD to KES, EUR to KES, etc.)
exchange_rates = {
    "USD": 128.88,
    "EUR": 140.57,
    "GBP": 167.27,
    "CAD": 94.32,
    "AUD": 87.20,
    "SGD": 96.73
    # Add other currencies and their rates here
}

# Convert mean salary to Kenyan Shillings (KES)
df["Exchange_Rate"] = df["Currency"].map(exchange_rates)
df["Mean_Salary_KES(^3)"] = df["Avg_Salary(e3)"] * df["Exchange_Rate"]

# Create a mapping of currency codes to country names
currency_to_country = {
    "USD": "United States",
    "GBP": "United Kingdom",
    "AUD": "Australia",
    "CAD": "Canada",
    "EUR": "Europe",
    "SGD": "Singapore"
    # Add other mappings as needed
}
df["Country"] = df["Currency"].map(currency_to_country)

# Replace "N/A" with NaN
df.replace("N/A", np.nan, inplace=True)


In [ ]:
# Move the 'TotalPrice' column to come after the 'UnitPrice' column

cols = list(df.columns)
cols.remove("Country")  # Remove 'Country' from the list
cols.insert(cols.index("Company") + 1, "Country")  # Insert 'Country' after 'Company'
df = df[cols]  # Reorder the columns

# Display the resulting DataFrame
#print(df)


In [ ]:
# Linearly interpolate missing values for Lower and Upper Salaries based on Currency
df['Lower_Salary'] = df.groupby('Currency')['Lower_Salary'].transform(lambda x: x.interpolate())
df['Upper_Salary'] = df.groupby('Currency')['Upper_Salary'].transform(lambda x: x.interpolate())
print("Linearly interpolated mean salaries:")
df.tail()

In [ ]:
df.head()

### Analysis and Visualizations

In [ ]:
# Checking for  missing Values
df.isnull()

In [ ]:
# Checking for missing Values
df.notnull()

### Distribution of Mean Salary based on Type of Country

In [ ]:
# Assuming your DataFrame is named 'df' with columns 'Country' and 'Salary'
df_clean = df.dropna(subset=['Mean_Salary_KES(^3)'])

# Calculate average salary by country
mean_salary_by_country = df_clean.groupby('Country')['Mean_Salary_KES(^3)'].mean()

# Set a muted color palette
colors = sns.color_palette("muted")

# Set the theme to 'darkgrid'
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))

# Create a bar plot for average salary by country
sns.barplot(x=mean_salary_by_country.index, y=mean_salary_by_country.values,  palette=colors, hue=mean_salary_by_country.index)
plt.xlabel('Country')
plt.ylabel('Average Salary')
plt.title('Average Salary by Country')

plt.show()


#### Average Salaries by Country and Experience

In [ ]:
df_clean = df.dropna(subset=['Mean_Salary_KES(^3)'])

# Calculate average salary by country and experience
mean_salary_by_country_exp = df_clean.groupby(['Country', 'Experience'])['Mean_Salary_KES(^3)'].mean().reset_index()

# Set a muted color palette
colors = sns.color_palette("muted")

# Set the theme to 'darkgrid'
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))

# Create a bar plot for average salary by country and experience
sns.barplot(x='Country', y='Mean_Salary_KES(^3)', hue='Experience', data=mean_salary_by_country_exp, palette=colors)
plt.xlabel('Country')
plt.ylabel('Average Salary')
plt.title('Average Salary by Country and Experience')

plt.show()



#### The Average Salaries by Jobe Type and Experience

In [ ]:
# Visulaizing The Average Salaries by Country and Experience
df_clean = df.dropna(subset=['Mean_Salary_KES(^3)'])

# Calculate average salary by country and experience
mean_salary_by_country_exp = df_clean.groupby(['Country', 'Job_type'])['Mean_Salary_KES(^3)'].mean().reset_index()

# Set a muted color palette
colors = sns.color_palette("Set3")

# Set the theme to 'darkgrid'
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))

# Create a bar plot for average salary by country and experience
sns.barplot(x='Country', y='Mean_Salary_KES(^3)', hue='Job_type', data=mean_salary_by_country_exp, palette=colors)
plt.xlabel('Country')
plt.ylabel('Average Salary')
plt.title('Average Salary by Job Type and Country')

plt.show()

### Findings and Recommendations

#### Findings

1. The European Countries are not specified apart from United Kingdom.
2. Only full time jobs are available
3. Most Countries have jobs in one or two level(s) of experience

#### Recommendations

1. Source for different levels of experience in order to attract more traffic and clients seeking employment opportunities
2. Specify the European Country where the job is located to make people understand issues form of communication expected to determine their suitability
3. Source for other types of jobs to encourage applications of suitable candidates that may need a=some level of flexibility 